In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

# Spotify


In [7]:
client_id = str(getpass.getpass('client_id?'))
client_sectret = str(getpass.getpass('client_secret?'))

In [8]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id, client_secret=client_sectret))

# Functions

In [18]:
def artists_from_playlist(playlist):
    songs = playlist['items']
    while playlist['next']:
        playlist = sp.next(playlist)
        songs.extend(playlist['items'])
    artists = []
    for item in songs:
        artists.append(item['track']['artists'][0]['name'])
    artist_id_list = []
    for artist in artists:
        artist_id = sp.search(artist)['tracks']['items'][0]['artists'][0]['id']
        artist_id_list.append(artist_id)
    return list(set(artist_id_list))

In [19]:
def all_albums_from_artists(artists_id_list):
    albums_ids = []
    for artist_id in artists_id_list:
        albums = sp.artist_albums(artist_id)
        for album in range(len(albums['items'])):
            albums_ids.append(albums['items'][album]['id'])
    return list(set(albums_ids))

In [20]:
def all_songs_from_albums(albums_id_list):
    ids = []
    for album_id in tqdm(albums_id_list):
        songs_ids = sp.album_tracks(album_id)
        for id in range(len(songs_ids['items'])):
            ids.append(songs_ids['items'][id]['id'])         
    ids_chunks = []
    for i in tqdm(range(0, len(ids), 100)):
        ids_chunks.append(ids[i:i + 100])
    audio_feat = []
    for i in tqdm(ids_chunks):
        audio_feat.append(sp.audio_features(tracks=i))
    audio_feat_total = []
    for i in tqdm(range(len(audio_feat))):
        audio_feat_total += audio_feat[i]
    audio_feats_clean = [dct for dct in audio_feat_total if dct]
    df = pd.DataFrame(audio_feats_clean)
    return df.drop(['type', 'track_href', 'uri', 'analysis_url'], axis=1)
    

In [21]:
def get_track_id(song):
    id_search = sp.search(song)
    if len(id_search['tracks']['items']) > 0:
        print()
        return id_search['tracks']['items'][0]['id']
    else:
        print('Typo? Try again')

In [22]:
def get_audio_features(song_id):
    a_f = sp.audio_features(tracks=song_id)
    if a_f:
        df = pd.DataFrame(a_f)
        return df.drop(['type', 'track_href', 'uri', 'analysis_url', 'id'], axis=1)
    else:
        return print('no features :(..')

In [5]:
def release_year(df):
    years = []
    for id in tqdm(df['id']):
        track = sp.track(id)
        years.append(track['album']['release_date'][0:4])
    return years

In [24]:
def scrape(playlist, name):
    artists = artists_from_playlist(playlist)
    albums = all_albums_from_artists(artists)
    songs = all_songs_from_albums(albums)
    songs.to_csv(name + '.csv', index=False)

# Playlists

In [57]:
rock_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DWXRqgorJj26U')

In [58]:
hard_rock_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DX68H8ZujdnN7')

In [59]:
jazz_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DXbITWG1ZJKYt')

In [60]:
jazz_cool_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DXbOVU4mpMJjh')

In [64]:
pop_playlist = sp.user_playlist_tracks('spotify', '3ZgmfR6lsnCwdffZUan8EA')

In [65]:
hiphop_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DX30w0JtSIv4j')

In [66]:
rap_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DX48TTZL62Yht')

In [67]:
trap_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DXde9tuMHuIsj')

In [68]:
deephouse_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DWWEJlAGA9gs0')

In [69]:
techno_playlist = sp.user_playlist_tracks('spotify', '37i9dQZF1DX6J5NfMJS675')

# Scraping

In [70]:
scrape(rock_playlist, 'rock_songs')

In [ ]:
scrape(hard_rock_playlist, 'hard_rock_songs')

In [ ]:
scrape(jazz_playlist, 'jazz_songs')

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
scrape(jazz_cool_playlist, 'jazz_cool_songs')

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
scrape(pop_playlist, 'pop_songs')

  0%|          | 0/2707 [00:00<?, ?it/s]

  0%|          | 0/283 [00:00<?, ?it/s]

  0%|          | 0/283 [00:00<?, ?it/s]

  0%|          | 0/283 [00:00<?, ?it/s]

In [ ]:
scrape(hiphop_playlist, 'hiphop_songs')

  0%|          | 0/696 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
scrape(rap_playlist, 'rap_songs')

  0%|          | 0/1260 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

In [ ]:
scrape(deephouse_playlist, 'deephouse_songs')

  0%|          | 0/1727 [00:00<?, ?it/s]

  0%|          | 0/339 [00:00<?, ?it/s]

  0%|          | 0/339 [00:00<?, ?it/s]

  0%|          | 0/339 [00:00<?, ?it/s]

In [ ]:
scrape(techno_playlist, 'techno_songs')

  0%|          | 0/942 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
scrape(trap_playlist, 'trap_songs')

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
genres_dict = {'jazz': 1 , 'rock': 2, 'hiphop':3, 'pop': 4, 'electronic': 5} 

# Wrangling & Cleaning

In [6]:
jazz = pd.read_csv('data/jazz_songs.csv')

In [7]:
jazz_cool = pd.read_csv('data/jazz_cool_songs.csv')

In [8]:
rock = pd.read_csv('data/rock_songs.csv')

In [9]:
hard_rock = pd.read_csv('data/hard_rock_songs.csv')

In [10]:
hiphop = pd.read_csv('data/hiphop_songs.csv')

In [11]:
rap = pd.read_csv('data/rap_songs.csv')

In [12]:
trap = pd.read_csv('data/trap_songs.csv')

In [13]:
pop = pd.read_csv('data/pop_songs.csv')

In [14]:
techno = pd.read_csv('data/techno_songs.csv')

In [15]:
deep = pd.read_csv('data/deephouse_songs.csv')

## Genres and release years

In [16]:
jazz['genre'] = 1
jazz['year'] = release_year(jazz)

  0%|          | 0/23686 [00:00<?, ?it/s]

In [17]:
jazz_cool['genre'] = 1
jazz_cool['year'] = release_year(jazz_cool)

  0%|          | 0/3198 [00:00<?, ?it/s]

In [23]:
rock['genre'] = 2
rock['year'] = release_year(rock)

  0%|          | 0/23458 [00:00<?, ?it/s]

In [27]:
hard_rock['genre'] = 2
hard_rock['year'] = release_year(hard_rock)

  0%|          | 0/14533 [00:00<?, ?it/s]

In [29]:
hiphop['genre'] = 3
hiphop['year'] = release_year(hiphop)

  0%|          | 0/8797 [00:00<?, ?it/s]

In [30]:
rap['genre'] = 3
rap['year'] = release_year(rap)

  0%|          | 0/10241 [00:00<?, ?it/s]

In [31]:
trap['genre'] = 3
trap['year'] = release_year(trap)

  0%|          | 0/2117 [00:00<?, ?it/s]

In [32]:
pop['genre'] = 4
pop['year'] = release_year(pop)

  0%|          | 0/28274 [00:00<?, ?it/s]

In [33]:
techno['genre'] = 5
techno['year'] = release_year(techno)

  0%|          | 0/4202 [00:00<?, ?it/s]

In [34]:
deep['genre'] = 5
deep['year'] = release_year(deep)

  0%|          | 0/33792 [00:00<?, ?it/s]

## Making the main DataFrame

In [35]:
music = pd.concat([jazz, jazz_cool, rock, hard_rock, hiphop, rap, trap, pop, techno, deep], axis=0, ignore_index=True)

In [36]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152298 entries, 0 to 152297
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      152298 non-null  float64
 1   energy            152298 non-null  float64
 2   key               152298 non-null  int64  
 3   loudness          152298 non-null  float64
 4   mode              152298 non-null  int64  
 5   speechiness       152298 non-null  float64
 6   acousticness      152298 non-null  float64
 7   instrumentalness  152298 non-null  float64
 8   liveness          152298 non-null  float64
 9   valence           152298 non-null  float64
 10  tempo             152298 non-null  float64
 11  id                152298 non-null  object 
 12  duration_ms       152298 non-null  int64  
 13  time_signature    152298 non-null  int64  
 14  genre             152298 non-null  int64  
 15  year              152298 non-null  object 
dtypes: float64(9), int64

In [37]:
music.tail()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,genre,year
152293,0.373,0.07400,1,-24.595,1,0.0509,0.745,0.8190,0.1020,0.1350,122.238,41PhuLu40MhT6vg3vr4778,375773,4,5,2021
152294,0.123,0.01280,2,-27.916,0,0.0401,0.692,0.2910,0.1090,0.0383,71.621,7Fa0w7GNcS57sMfTofiG4R,824013,4,5,2021
152295,0.342,0.02570,0,-31.669,1,0.0447,0.776,0.0956,0.1120,0.3620,158.439,4SniQ36o6fHpIKwHVNnJoN,305600,3,5,2021
152296,0.142,0.00248,7,-34.535,1,0.0476,0.270,0.0232,0.0596,0.0366,74.570,0lotD0m9lAMrmZZ85N9QDC,750187,4,5,2021
152297,0.165,0.09830,10,-24.322,1,0.0432,0.721,0.6430,0.3050,0.0461,72.211,1rlxhEmCV8D2eyL6rJwXLD,600280,4,5,2021


## Check and drop duplicates

In [38]:
music.duplicated(subset=['id']).value_counts()

False    142783
True       9515
dtype: int64

In [39]:
music.drop_duplicates(subset=['id'], inplace=True, ignore_index=True)

In [40]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142783 entries, 0 to 142782
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      142783 non-null  float64
 1   energy            142783 non-null  float64
 2   key               142783 non-null  int64  
 3   loudness          142783 non-null  float64
 4   mode              142783 non-null  int64  
 5   speechiness       142783 non-null  float64
 6   acousticness      142783 non-null  float64
 7   instrumentalness  142783 non-null  float64
 8   liveness          142783 non-null  float64
 9   valence           142783 non-null  float64
 10  tempo             142783 non-null  float64
 11  id                142783 non-null  object 
 12  duration_ms       142783 non-null  int64  
 13  time_signature    142783 non-null  int64  
 14  genre             142783 non-null  int64  
 15  year              142783 non-null  object 
dtypes: float64(9), int64

In [41]:
music.tail()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,genre,year
142778,0.373,0.07400,1,-24.595,1,0.0509,0.745,0.8190,0.1020,0.1350,122.238,41PhuLu40MhT6vg3vr4778,375773,4,5,2021
142779,0.123,0.01280,2,-27.916,0,0.0401,0.692,0.2910,0.1090,0.0383,71.621,7Fa0w7GNcS57sMfTofiG4R,824013,4,5,2021
142780,0.342,0.02570,0,-31.669,1,0.0447,0.776,0.0956,0.1120,0.3620,158.439,4SniQ36o6fHpIKwHVNnJoN,305600,3,5,2021
142781,0.142,0.00248,7,-34.535,1,0.0476,0.270,0.0232,0.0596,0.0366,74.570,0lotD0m9lAMrmZZ85N9QDC,750187,4,5,2021
142782,0.165,0.09830,10,-24.322,1,0.0432,0.721,0.6430,0.3050,0.0461,72.211,1rlxhEmCV8D2eyL6rJwXLD,600280,4,5,2021


In [42]:
music.to_csv('music.csv', index=False)

In [60]:
music['year'].value_counts()

2021    27730
2020    20319
2019    13010
2018     7493
2017     7104
        ...  
1958       35
1960       24
0000       24
1954       11
1956        6
Name: year, Length: 68, dtype: int64